In [ ]:
import pandas as pd
from pathlib import Path
import numpy as np

## Fatura - Aba principal

In [ ]:
raw_data_path = Path("../data/raw/CPFL-RS-Informações industrial grupo A.xlsx").resolve()

processed_data_folder = Path("../data/processed").resolve()
processed_data_folder.mkdir(parents=True, exist_ok=True)

data = pd.read_excel(raw_data_path)
data.head(30)

In [ ]:
columns_vars = data.iloc[3,0:7].values
columns_months = data.iloc[2,7:19].values
data_columns = np.append(columns_vars, columns_months)

data_consumption = data.iloc[:,0:19]
data_consumption.columns = data_columns
data_consumption = data_consumption.drop([0,1,2,3])
data_consumption = data_consumption.melt(columns_vars).dropna(subset="value")
data_consumption["value"] = pd.to_numeric(data_consumption["value"])
data_consumption["grupo"] = "consumo"

data_demand = pd.concat([data.iloc[:,0:7], data.iloc[:,19:31]], axis=1)
data_demand.columns = data_columns
data_demand = data_demand.drop([0,1,2,3])
data_demand = data_demand.melt(columns_vars).dropna(subset="value")
data_demand["value"] = pd.to_numeric(data_demand["value"])
data_demand["grupo"] = "demanda"

data_fer = pd.concat([data.iloc[:,0:7], data.iloc[:,31:]], axis=1)
data_fer.columns = data_columns
data_fer = data_fer.drop([0,1,2,3])
data_fer = data_fer.melt(columns_vars).dropna(subset="value")
data_fer["value"] = pd.to_numeric(data_fer["value"])
data_fer["grupo"] = "fer"

data_all = pd.concat(
    [data_consumption, data_demand, data_fer],
    axis=0
)

data_all.rename(columns={"variable": "Meses", "value": "Valores"}, inplace=True)
data_all

## Padrões de clientes

In [ ]:
client_data = data_all[["Nº instalação", "Classe de cálculo", "SubGrupo de Tensão", "Município"]]
client_data = client_data.groupby("Nº instalação").max().reset_index()
client_data.to_csv("../data/processed/clientes.csv")

## Energia Injetada

In [ ]:
data_inj = pd.read_excel(raw_data_path, sheet_name="Energia injetada")
data_inj

In [ ]:
columns_vars = data_inj.iloc[3,0:2].values
columns_months = data_inj.iloc[2,2:14].values
data_columns = np.append(columns_vars, columns_months)

data_inj.columns = data_columns
data_inj["Item Faturamento BW"] = "InjetadaTrue"
data_inj["SubAgrupador de Fatu"] = "Injetada"
data_inj["grupo"] = "injetada"
columns_vars = np.append(columns_vars, "Item Faturamento BW")
columns_vars = np.append(columns_vars, "SubAgrupador de Fatu")
columns_vars = np.append(columns_vars, "grupo")
data_inj = data_inj.drop([0,1,2,3])
data_inj = data_inj.melt(columns_vars, value_vars=columns_months)
data_inj.rename(columns={"variable": "Meses", "value": "Valores"}, inplace=True)

data_inj = pd.merge(data_inj, client_data, how="left", on="Nº instalação")

data_inj

In [ ]:
data_all = pd.concat(
    [data_all, data_inj],
    axis=0
)
data_all

In [ ]:
file_save_path = processed_data_folder / f"consumo_tarifas_meses.xlsx"
data_all.to_excel(file_save_path, index=False)

## Demanda Contratada

In [ ]:
data_cont = pd.read_excel(raw_data_path, sheet_name="Demanda contratada")
data_cont

In [ ]:
# Process Tipo
data_cont[["Tipo", "Tipo_THS"]] = data_cont["Tipo"].str.split("THS", expand= True)
data_cont["Tipo"] = data_cont["Tipo"].str.replace(pat=":", repl="")
# Process Operando
data_cont[["Operando", "Operando_THS"]] = data_cont["Operando"].str.split("THS", expand=True)
data_cont["Operando"] = data_cont["Operando"].str.replace(pat=":", repl="")
# Process Classificação GD
data_cont["Classificação GD"] = data_cont["Classificação GD"].apply(lambda x: str(x)[4:-1] if x != "-" else x)
# Get Dummies from Classificação GD
data_cont = pd.get_dummies(data_cont, columns=["Classificação GD"], prefix="GD")
data_cont.loc[data_cont["GD_Geradora&Beneficiária"], ["GD_Beneficiária", "GD_Geradora"]] = True
data_cont = data_cont.drop(["GD_Geradora&Beneficiária", "GD_-"], axis=1)
data_cont

In [ ]:
file_save_path = processed_data_folder / f"cadastro_clientes_demanda.xlsx"
data_cont.to_excel(file_save_path, index=False)